In [1]:
from agents import Agent, function_tool, Runner

In [2]:
from agents import Runner

In [3]:
runner = Runner()

In [4]:
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIAgentsSDKRunner

chat_interface = IPythonChatInterface()

runner = OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=web_agent
)


## Summary Agent

In [7]:
import youtube

In [11]:
def fetch_youtube_transcript(video_id: str) -> str:
    """
    Fetches the transcript of a YouTube video and converts it into a subtitle-formatted string.

    Args:
        video_id (str): The unique YouTube video ID.

    Returns:
        str: The subtitles generated from the video's transcript.
    """
    return youtube.fetch_transcript_cached(video_id)

In [12]:
summary_instructions = """
You're a helpful assistant that summarizes youtube videos
"""

In [18]:
summary_agent = Agent(
    name='summary_agent',
    instructions=summary_instructions,
    handoff_description="Whenever the user needs a summary of the video",
    tools=[function_tool(fetch_youtube_transcript)],
    model='gpt-4o-mini'
)

## Search Agent

In [20]:
from minsearch import AppendableIndex

In [54]:
index = AppendableIndex(text_fields=['content'])

In [21]:
import docs

In [69]:
from typing import Any, Dict, List, Optional
from requests.exceptions import RequestException


class SearchTools:

    def __init__(self, index: Any) -> None:
        self.index = index

    def index_video(self, video_id: str, content: Optional[str] = None) -> bool:
        """
        Fetch and index the transcript of a YouTube video.

        Args:
            video_id (str): The unique identifier of the YouTube video.

        Returns:
            bool: True if indexing succeeds, False otherwise.
        """
        try:
            if not content:
                content = youtube.fetch_transcript_cached(video_id)
                if not content:
                    print(f"No subtitles found for video: {video_id}")
                    return False

            chunks = docs.sliding_window(content, 2000, 1000)
            for chunk in chunks:
                chunk["video_id"] = video_id
                self.index.append(chunk)

            return True

        except (RequestException, ValueError) as e:
            print(f"Error indexing video {video_id}: {e}")
            return False
        except AttributeError:
            print("Error: 'youtube' or 'docs' module not properly configured.")
            return False
        except Exception as e:
            print(f"Unexpected error during indexing: {e}")
            return False

    def search(self, query: str) -> Dict[str, Any]:
        """
        Search for relevant results in the indexed video transcripts.

        Args:
            query (str): The user's search query.

        Returns:
            Dict[str, Any]: A dictionary containing the top search results.
        """
        if not isinstance(query, str) or not query.strip():
            raise ValueError("Query must be a non-empty string.")

        try:
            results = self.index.search(query, num_results=5)
            return results
        except AttributeError:
            raise AttributeError("Index object must implement a 'search(query, num_results=...)' method.")
        except Exception as e:
            print(f"Error performing search: {e}")
            return {}


In [70]:
search_tools = SearchTools(index)

In [56]:
from toyaikit.tools import wrap_instance_methods

In [57]:
search_method_tools = wrap_instance_methods(function_tool, search_tools)

In [58]:
search_instructions = """
Your task is to search through indexed documents.

Before performing a search:

1. Check if the document has been indexed.
2. If not, call `index_video` to index it.

Important rules:

- Do NOT call `search` using a YouTube ID, webpage URL, or any direct link as the query.
- After successfully indexing a source, do NOT perform a search automatically.
- Instead, tell the user something like:
  "The video (or page) has been indexed. What would you like to learn about it?"

Only perform a search if the user provides a natural-language question or topic of interest
(e.g., "What does the video say about AI safety?"), not a URL or ID.

Do not attempt to summarize content.
"""

search_agent = Agent(
    name='search_agent',
    instructions=summary_instructions,
    handoff_description="Whenever the user needs to search for things about videos",
    tools=search_method_tools,
    model='gpt-4o-mini'
)

In [46]:
runner = OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=search_agent
)

## Summary Agent Updated

In [7]:
import youtube

In [77]:
class SummaryTools:

    def __init__(self, search_tools):
        self.search_tools = search_tools

    def fetch_youtube_transcript(self, video_id: str) -> str:
        """
        Fetches the transcript of a YouTube video and converts it into a subtitle-formatted string.
    
        Args:
            video_id (str): The unique YouTube video ID.
    
        Returns:
            str: The subtitles generated from the video's transcript.
        """
        content = youtube.fetch_transcript_cached(video_id)
        self.search_tools.index_video(video_id, content)
        return content

In [78]:
summary_tools = SummaryTools(search_tools)

In [79]:
summary_instructions = """
You're a helpful assistant that summarizes youtube videos
"""

In [80]:
summary_agent = Agent(
    name='summary_agent',
    instructions=summary_instructions,
    handoff_description="Whenever the user needs a summary of the video",
    tools=[function_tool(summary_tools.fetch_youtube_transcript)],
    model='gpt-4o-mini'
)

## Triage / Orchestrator

In [81]:
from agents import handoff

In [82]:
triage_instructions = """
You are the orchestrator between two specialized agents:

1. summarizing_agent — summarizes web pages or YouTube videos.
2. search_agent — searches within previously indexed documents to answer detailed or follow-up questions.

Routing rules:

- If the user sends a YouTube link without any question → hand off to summary_agent.
- If the user asks for a summary, overview, or "what is this video about" → summary_agent.
- If the user asks a follow-up question after a summary (e.g. "how did they do X?", "what does she say about Y?"),
  or refers to something mentioned in a previously summarized or indexed resource → hand off to search_agent.
- If the user asks a direct content question about a topic inside a resource → search_agent.
- Prefer delegating the answer to agents when possible. 

Examples:
User: "https://youtube.com/watch?v=abc123"
→ summary_agent

User: "What does the video say about climate change?"
→ search_agent

User: "How exactly did they fight malaria?" (after a summary)
→ search_agent
""".strip()

triage_agent = Agent(
    name='triage_agent',
    instructions=triage_instructions,
    handoffs=[
        handoff(summary_agent, on_handoff=lambda ctx: print('handoff to summary agent')),
        handoff(search_agent, on_handoff=lambda ctx: print('handoff to search agent')),
    ],
    model='gpt-4o-mini'
)

In [83]:
runner = OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=triage_agent
)

await runner.run();

You: gXvVMvhfrIY


handoff to summary agent
handoff: transfer_to_summary_agent
handoff: summary_agent -> triage_agent successful


You: tell me more about his work at bol


handoff to search agent
handoff: transfer_to_search_agent
handoff: search_agent -> triage_agent successful


You: stop


Chat ended.
